# 0.0 Imports

In [1]:
import pandas as pd
from pandas.core import apply
import re

# Primeira Visão


In [2]:
df_raw = pd.read_json('gdg_scrapy/gdg_home_products.json.json')

In [4]:
#Idenfiticar Problemas na importação dos dados (Percebe-se que o maior problema está nas cores)
df_raw.isna().sum()

url                     0
nome                    0
id                      0
cor                  7550
espec                   0
categoria               0
preco_antigo            0
preco_atual             0
preco_boleto            0
condicao_parcela        0
desconto_destaque    3203
dtype: int64

Por quê existem alguns produtos sem o valor de desconto em destaque?

In [17]:
#Filtrar produtos em que a Tag de descontos é nula
df_desconto = df_raw[['id',
                      'url',
                      'nome',
                      'categoria',
                      'preco_antigo',
                      'preco_atual',
                      'preco_boleto',
                      'desconto_destaque']]

desconto_na = df_desconto.loc[df_desconto['desconto_destaque'].isna()]
df.head()

,url,nome,id,cor,espec,categoria,preco_antigo,preco_atual,preco_boleto,condicao_parcela,desconto_destaque
0,https://www.graodegente.com.br/roupinhas-3/cas...,"Casaquinho Tricot Jacquard Geométrico Cinza, R...",168087,None,[Tamanho],Roupinhas,"de R$154,27","R$127,36",120.99,"\r\n 12x de R$12,42...",21%
1,https://www.graodegente.com.br/kit-berco/kit-b...,Kit Berço Trança Theodora Princesa,168086,Rosa,"[Número de Peças, Cor: , Rosa]",Kit Berço,"de R$133,32","R$106,66",101.33,"\r\n 12x de R$10,40...",23%
2,https://www.graodegente.com.br/kit-berco/kit-b...,Kit Berço Trança Teddy Príncipe Azul Marinho,168085,Azul Marinho,"[Número de Peças, Cor: , Azul Marinho]",Kit Berço,"de R$133,32","R$106,66",101.33,"\r\n 12x de R$10,40...",23%
3,https://www.graodegente.com.br/roupinhas-3/cas...,Casaquinho Tricot Vazado Rosa Tam G,168084,Rosa,"[Tamanho, Cor: , Rosa]",Roupinhas,"de R$154,27","R$127,36",120.99,"\r\n 12x de R$12,42...",21%
4,https://www.graodegente.com.br/kit-berco/kit-b...,Kit Berço Trança Teddy Príncipe Azul,168083,Azul,"[Número de Peças, Cor: , Azul]",Kit Berço,"de R$133,32","R$106,66",101.33,"\r\n 12x de R$10,40...",23%


Existe diferença entre o preço antigo e o preço novo de produtos sem a tag de desconto, entretanto, preciso verificar o motivo.

### Hipótese:
 O valor do desconto é muito baixo para chamar a atenção dos compradores.

# 3 Limpeza dos Dados

In [106]:
df = df_raw.copy()

In [107]:
#Remover dados unitários do tipo lista
def remove_lista(df, df_columns):
  for column in df_columns: 
    df[column] =  df[column].apply(lambda x: ''.join(x))
    df[column].astype(str)
  return df

In [108]:
df2 = remove_lista(df, df_columns=['id', 'categoria', 'preco_boleto'])
df2 = df2[['id', 'nome', 'categoria', 'cor', 'preco_atual', 'preco_antigo', 'preco_boleto', 'condicao_parcela', 'desconto_destaque','url']]

In [109]:
#Expressões regulares para tratar o preço
re_preco = '\d.*\,\d*'
re_parcela_valor = '\d*\,\d*'
re_parcela = '(\d*)x'

In [110]:
#Preço atual - (Por alguma ração, o notebook não me deixa subtituir a coluna principal)
df2['valor_atual'] = df2['preco_atual'].apply(lambda x: re.search(re_preco, x).group(0))
df2['valor_riscado'] = df2['preco_antigo'].apply(lambda x: re.search(re_preco, x).group(0))
df2['parcela_valor'] = df2['condicao_parcela'].apply(lambda x: re.search(re_parcela_valor, x).group(0))
df2['parcela_qtd'] = df2['condicao_parcela'].apply(lambda x: re.search(re_parcela, x).group(0))

In [111]:
df2.head()

,id,nome,categoria,cor,preco_atual,preco_antigo,preco_boleto,condicao_parcela,desconto_destaque,url,valor_atual,valor_riscado,parcela_valor,parcela_qtd
0,168087,"Casaquinho Tricot Jacquard Geométrico Cinza, R...",Roupinhas,None,"R$127,36","de R$154,27",120.99,"\r\n 12x de R$12,42...",21%,https://www.graodegente.com.br/roupinhas-3/cas...,"127,36","154,27","12,42",12x
1,168086,Kit Berço Trança Theodora Princesa,Kit Berço,Rosa,"R$106,66","de R$133,32",101.33,"\r\n 12x de R$10,40...",23%,https://www.graodegente.com.br/kit-berco/kit-b...,"106,66","133,32","10,40",12x
2,168085,Kit Berço Trança Teddy Príncipe Azul Marinho,Kit Berço,Azul Marinho,"R$106,66","de R$133,32",101.33,"\r\n 12x de R$10,40...",23%,https://www.graodegente.com.br/kit-berco/kit-b...,"106,66","133,32","10,40",12x
3,168084,Casaquinho Tricot Vazado Rosa Tam G,Roupinhas,Rosa,"R$127,36","de R$154,27",120.99,"\r\n 12x de R$12,42...",21%,https://www.graodegente.com.br/roupinhas-3/cas...,"127,36","154,27","12,42",12x
4,168083,Kit Berço Trança Teddy Príncipe Azul,Kit Berço,Azul,"R$106,66","de R$133,32",101.33,"\r\n 12x de R$10,40...",23%,https://www.graodegente.com.br/kit-berco/kit-b...,"106,66","133,32","10,40",12x


In [112]:
#Remove caracteres especiais (Ainda Não usado)
def remover_caracteres(df, columns, old, new ):
  for column in columns:
    df[column] = df[columns].apply(lambda x: x.replace(old, new))
  return df     

In [114]:
#Remover caracteres especiais


df2['valor_atual'] = df2['valor_atual'].apply(lambda x: x.replace('.', ''))
df2['valor_atual'] = df2['valor_atual'].apply(lambda x: x.replace(',', '.'))

df2['valor_riscado'] = df2['valor_riscado'].apply(lambda x: x.replace('.', ''))
df2['valor_riscado'] = df2['valor_riscado'].apply(lambda x: x.replace(',', '.'))

df2['parcela_valor'] = df2['parcela_valor'].apply(lambda x: x.replace('.', ''))
df2['parcela_valor'] = df2['parcela_valor'].apply(lambda x: x.replace(',', '.'))


df2['parcela_qtd'] = df2['parcela_qtd'].apply(lambda x: x.replace('x', ''))

df2['desconto_destaque'] = df2['desconto_destaque'].astype(str)
df2['desconto'] = df2['desconto_destaque'].apply(lambda x: x.replace('%', ''))

In [123]:
# Identifica os erros  e transforma em porcentagem no formato decimal
df2['desconto'] = df2['desconto'].apply(lambda x: x if x != 'None' else None )
df2['desconto'] = df2['desconto'].astype('float64').apply(lambda x: x / 100)

In [124]:
#Feature Engeneereing

# valor_total_parcela
df2['total_parcela'] = df2['parcela_valor'].astype(float) * df2['parcela_qtd'].astype(int)
df2['item_id'] = df2['id'].apply(lambda x: x[:4])
df2['spec_id'] = df2['id'].apply(lambda x: x[4:])

In [125]:
df_final = df2[['id',
                'item_id',
                'spec_id',
                'nome',
                'categoria',
                'url',
                'valor_atual',
                'valor_riscado',
                'preco_boleto',
                'parcela_valor',
                'parcela_qtd',
                'total_parcela',
                'desconto']]

In [126]:
df_final.head()

,id,item_id,spec_id,nome,categoria,url,valor_atual,valor_riscado,preco_boleto,parcela_valor,parcela_qtd,total_parcela,desconto
0,168087,1680,87,"Casaquinho Tricot Jacquard Geométrico Cinza, R...",Roupinhas,https://www.graodegente.com.br/roupinhas-3/cas...,127.36,154.27,120.99,12.42,12,149.04,0.0021
1,168086,1680,86,Kit Berço Trança Theodora Princesa,Kit Berço,https://www.graodegente.com.br/kit-berco/kit-b...,106.66,133.32,101.33,10.40,12,124.80,0.0023
2,168085,1680,85,Kit Berço Trança Teddy Príncipe Azul Marinho,Kit Berço,https://www.graodegente.com.br/kit-berco/kit-b...,106.66,133.32,101.33,10.40,12,124.80,0.0023
3,168084,1680,84,Casaquinho Tricot Vazado Rosa Tam G,Roupinhas,https://www.graodegente.com.br/roupinhas-3/cas...,127.36,154.27,120.99,12.42,12,149.04,0.0021
4,168083,1680,83,Kit Berço Trança Teddy Príncipe Azul,Kit Berço,https://www.graodegente.com.br/kit-berco/kit-b...,106.66,133.32,101.33,10.40,12,124.80,0.0023


In [ ]:
df_final.to_csv('products_beta.csv', index=False)